In [1]:
import pandas as pd
from __future__ import division

pd.set_option('display.max_columns',100)
pd.set_option('display.max_rows',200)

In [51]:
df = pd.read_csv('./../working-data/bzn-biz-licenses.csv')

In [63]:
# Concat to single address field
def str_not_nan(value):
    if pd.isnull(value):
        return ""
    else:
        return str(value)
keys = ['street_num', 'street_dir_code', 'street_name', 'street_suffix', 'street_post_qual', 'street_apt_num']

df['address'] = df[keys].apply(lambda x: ' '.join(x.map(str_not_nan)), axis=1)


In [67]:
print df.columns
df.head()

Index([u'biz_name', u'license_num', u'control_num', u'resp_dept_code',
       u'resp_dept_descr', u'class_code', u'class_descr', u'fiscal_year',
       u'license_status', u'code_descr', u'status_date', u'application_date',
       u'issue_date', u'licen_exp_date', u'renew_date', u'location_id',
       u'street_num', u'street_pre_qual', u'street_dir_code', u'street_name',
       u'street_suffix', u'street_post_qual', u'street_apt_num',
       u'class_desc_cat', u'class_desc_detail', u'address'],
      dtype='object')


,biz_name,license_num,control_num,resp_dept_code,resp_dept_descr,class_code,class_descr,fiscal_year,license_status,code_descr,status_date,application_date,issue_date,licen_exp_date,renew_date,location_id,street_num,street_pre_qual,street_dir_code,street_name,street_suffix,street_post_qual,street_apt_num,class_desc_cat,class_desc_detail,address
0,... AND SEW ON,52085,25732,BL,BUSINESS LICENSE,SER 28,SERVICE - SEWING/ALTERATIONS,15,IN,INACTIVE,2015-12-16,2015-01-22,2015-08-02,2015-12-31,NaN,52760,601,NaN,NaN,NIKLES,DR,NaN,NaN,SERVICE,SEWING/ALTERATIONS,601 NIKLES DR
1,... AND SEW ON,53948,25732,BL,BUSINESS LICENSE,RET 38,RETAIL - OTHER,16,AC,ACTIVE,2016-03-02,2015-12-16,2016-03-02,2016-12-31,NaN,52760,601,NaN,NaN,NIKLES,DR,NaN,NaN,RETAIL,OTHER,601 NIKLES DR
2,@ YOUR SERVICE,53148,26432,BL,BUSINESS LICENSE,SER 31,SERVICE - OTHER,15,RN,RENEWED,2016-01-13,2015-07-28,2015-10-12,2015-12-31,2016-01-13,84770,100,NaN,NaN,OUT OF CITY LIMITS,ST,NaN,NaN,SERVICE,OTHER,100 OUT OF CITY LIMITS ST
3,@ YOUR SERVICE,53148,26432,BL,BUSINESS LICENSE,SER 31,SERVICE - OTHER,16,AC,ACTIVE,2016-01-13,2016-01-13,2016-01-13,2016-12-31,NaN,84770,100,NaN,NaN,OUT OF CITY LIMITS,ST,NaN,NaN,SERVICE,OTHER,100 OUT OF CITY LIMITS ST
4,"""DO ME A PRETTY FAVOR PLEASE!""",19250,12322,BL,BUSINESS LICENSE,SER 31,SERVICE - OTHER,7,IN,INACTIVE,2008-04-09,2007-04-19,2007-04-19,2007-12-31,NaN,84770,100,NaN,NaN,OUT OF CITY LIMITS,ST,NaN,NaN,SERVICE,OTHER,100 OUT OF CITY LIMITS ST


In [ ]:
# df_16 = df[df['fiscal_year'] == 16]

In [ ]:
# len(df_16)

In [ ]:
# years = df['fiscal_year'].unique()

# prev_year = 2823
# for year in range(2, 16):
#     num = len(df[df['fiscal_year'] == year])
#     print year, num, num / prev_year * 100 - 100
#     prev_year = num

In [ ]:
# df_16['class_descr'].unique()

In [ ]:
# grouped = df_16.groupby(['class_desc_cat', 'class_desc_detail']).agg({'license_num': 'count'})
# grouped.reset_index().sort_values(['class_desc_cat', 'license_num'], ascending=[True, False])
# # SEEMS TO DROP BUSINESSES W/ NaN class_desc_detail

# df_16['class_descr'].value_counts()

In [ ]:
# df_16[df_16['class_descr'] == 'SERVICE - OTHER']

In [69]:
main_street = df[df['street_name'].str.contains('MAIN') & df['code_descr'].isin(['RENEWED', 'ACTIVE'])]

In [ ]:
main_street['location_id'].value_counts()

In [ ]:
len(main_street['location_id'].unique())

In [ ]:
main_street[main_street['location_id'] == 66510]

In [ ]:
main_street['code_descr'].unique()

In [72]:
aggregate = {
    'issue_date': {'first_date': min, 'last_date': max, 'renewals': len},
    'class_desc_cat': min,
    'class_desc_detail': min
}

by_location = main_street.groupby(['address','location_id','biz_name']).agg(aggregate)

In [73]:
by_location

issue_date  \
                                                                                   last_date   
address                location_id biz_name                                                    
1 E MAIN ST            26600       BLACK MOUNTAIN ARCHITECTURE LLC                2016-03-11   
                                   BRIDGER KITCHEN INTERIORS, INC                 2010-12-30   
                                   GIRLS OUTDOORS                                 2016-02-18   
                                   PLUM DESIGN LAB LLC                            2016-03-10   
10 E MAIN ST           100000      BETTER BODY FITNESS OF MT INC                  2008-12-24   
                                   BOZEMAN CAMERA & REPAIR,INC                    2015-12-21   
                                   COSMICA INC                                    2015-12-18   
                                   ELIZABETH & CO                                 2010-04-16   
1006 W MAIN ST         17910       BOZEMAN URGENT CARE CENTER                     2013-03-21   
                                   MOUNTAIN SPRINGS RANCH N ASSOC                 2015-12-24   
1007 E MAIN ST         36070       AVITEL, INC                                    2015-12-21   
1007 W MAIN ST         100950      FARMSTEAD PIZZA LLC DBA RED TRACTOR PIZZA      2016-01-07   
                                   ON THE RISE                                    2009-02-26   
                                   SOLA PRESTA!                                   2013-02-28   
101 E MAIN ST UNIT 1   146620      THINKONE CONSTRUCTION MGMT                     2015-12-24   
101 E MAIN ST UNIT 2   146630      COPPER LLC                                     2015-12-15   
101 E MAIN ST UNIT A   26690       BIG SKY JOURNAL                                2016-01-29   
                                   FORTUNE PERSONNEL CONSULTANT                   2009-01-15   
                                   GUADALAJARA MEXICAN RESTAURANT                 2006-01-05   
                                   LANGLAS & ASSOC-INACTIVE                       2006-05-09   
                                   LOOIE'S DOWN UNDER/JADRA'S                     2010-02-22   
                                   MERIDIAN                                       2016-01-30   
                                   RAIN                                           2003-02-10   
                                   SWEET CHILI ASIAN BISTRO LLC                   2015-12-16   
101 E MAIN ST UNIT G   207850      HALLMARK BUSINESS CONNECTIONS                  2014-01-28   
101 W MAIN ST          26510       CHALET SPORTS                                  2016-01-30   
1010 E MAIN ST         66910       BLUE SKY MOTEL                                 2011-01-31   
                                   CS AND COMPANY                                 2016-01-04   
                                   FOUR POINTS LLC                                2014-10-14   
1015 W MAIN ST         66260       WENTANA LLC, DBA 15003                         2015-12-18   
1017 E MAIN ST         215260      ALPINE LODGE, LLC                              2002-02-21   
                                   ANDERSON ZURMUEHLEN & CO P.C.                  2016-01-07   
                                   FERGUSON ENTERPRISES INC                       2016-02-02   
                                   LANGLAS & ASSOCIATES, INC                      2015-12-21   
104 E MAIN ST          4870        ASPEN COUNSELING SERVICES                      2003-01-27   
                                   BOCKHOLT LANDSCAPE ARCHITECTURE                2016-01-26   
                                   BRENTWOOD SERVICES ADMINISTRATORS INC          2016-01-15   
                                   ED LEWIS CONSULTING                            2015-12-18   
                                   HIGH COUNTRY FINCL STRATEGIES                  2003-02-14   
                                   MONTANA INVESTM

In [76]:
by_location.to_excel('./../working-data/main-street-biz-by-location.xlsx')

In [74]:
by_loc_and_year = pd.pivot_table(main_street,
                                values='issue_date',
                                index=['address','biz_name','class_desc_cat','class_desc_detail'],
                                columns='fiscal_year',
                                aggfunc=min)


In [75]:
by_loc_and_year

fiscal_year                                                                                                                 2   \
address                biz_name                                      class_desc_cat class_desc_detail                            
1 E MAIN ST            BRIDGER KITCHEN INTERIORS, INC                RETAIL         FURNITURE/HOME INTERIORS        2002-01-02   
                       GIRLS OUTDOORS                                RETAIL         CLOTHING AND/OR SHOES                 None   
                       PLUM DESIGN LAB LLC                           SERVICE        OTHER                                 None   
10 E MAIN ST           BETTER BODY FITNESS OF MT INC                 RETAIL         OTHER                                 None   
                       BOZEMAN CAMERA & REPAIR,INC                   RETAIL         ELECTRONIC/ELECTRIC SALES             None   
                                                                     SERVICE        MAINTENANCE/REPAIR SVCS         2002-01-02   
                       COSMICA INC                                   RETAIL         CLOTHING AND/OR SHOES                 None   
                       ELIZABETH & CO                                RETAIL         CLOTHING AND/OR SHOES           2002-10-31   
1006 W MAIN ST         BOZEMAN URGENT CARE CENTER                    SERVICE        HEALTH CARE                     2002-02-07   
                       MOUNTAIN SPRINGS RANCH N ASSOC                CONTRACTORS    LAWN CARE & LANDSCAPING               None   
1007 E MAIN ST         AVITEL, INC                                   RETAIL         ELECTRONIC/ELECTRIC SALES       2002-01-23   
1007 W MAIN ST         FARMSTEAD PIZZA LLC DBA RED TRACTOR PIZZA     FOOD SERVICE   RESTAURANT                            None   
                       ON THE RISE                                   RETAIL         SPECIALTY FOOD                  2002-02-21   
                       SOLA PRESTA!                                  FOOD SERVICE   RESTAURANT                            None   
101 E MAIN ST UNIT 1   THINKONE CONSTRUCTION MGMT                    CONTRACTORS    GENERAL                               None   
101 E MAIN ST UNIT 2   COPPER LLC                                    FOOD SERVICE   RESTAURANT                            None   
101 E MAIN ST UNIT A   FORTUNE PERSONNEL CONSULTANT                  SERVICE        CONSULTING                      2002-02-04   
                       GUADALAJARA MEXICAN RESTAURANT                FOOD SERVICE   RESTAURANT                            None   
                       LANGLAS & ASSOC-INACTIVE                      CONTRACTORS    GENERAL                               None   
                       LOOIE'S DOWN UNDER/JADRA'S                    FOOD SERVICE   RESTAURANT                      2002-02-19   
                       MERIDIAN                                      RETAIL         CLOTHING AND/OR SHOES           2004-04-30   
                       RAIN                                          FOOD SERVICE   RESTAURANT                      2002-02-19   
                       SWEET CHILI ASIAN BISTRO LLC                  FOOD SERVICE   RESTAURANT                            None   
101 E MAIN ST UNIT G   HALLMARK BUSINESS CONNECTIONS                 SERVICE        COMPUTER CONSULTING/SYS DESIGN        None   
101 W MAIN ST          CHALET SPORTS                                 RETAIL         SPORTS EQUIP/SPORTING GOODS     2002-01-23   
1010 E MAIN ST         CS AND COMPANY                                SERVICE        BOOKKEEPING                           None   
                       FOUR POINTS LLC                               CONTRACTORS    GENERAL                               None   
1015 W MAIN ST         WENTANA LLC, DBA 15003                        FOOD SERVICE   RESTAURANT                      2002-01-14   
1017 E MAIN ST         ANDERSON ZURMUEHLEN & CO P.C.                 SERVICE        BOOKKEEPING     

In [77]:
by_loc_and_year.to_excel('./../working-data/main-street-biz-by-loc-and-year.xlsx')

In [99]:
by_loc_and_year.to_csv('./../working-data/main-street-biz-by-loc-and-year.csv')